In [ ]:
# 0. Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1. Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import classification_report, roc_auc_score
from scipy.stats.mstats import winsorize
from imblearn.over_sampling import SMOTE

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# 2. Set Seed untuk Konsistensi
SEED = 44
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 3. Load Dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/KCCAE/KCCAEC2_dataset_latent.csv')
X = data.drop('Diabetes_012', axis=1)
y = data['Diabetes_012']

# 6. Split Data (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

# 7. SMOTE on Train Data Only
smote = SMOTE(random_state=SEED)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 7.1 Simpan Dataset Hasil SMOTE
df_train_smote = pd.DataFrame(X_train_smote, columns=X_train.columns)
df_train_smote['Diabetes_012'] = y_train_smote
df_train_smote.to_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/Output DNN+KCC+AE+SMOTE.csv', index=False)

# 8. One-hot Encoding
y_train_encoded = to_categorical(y_train_smote)
y_test_encoded = to_categorical(y_test)

# 9. Split Train → Train Main & Validation
X_train_main, X_val, y_train_main, y_val = train_test_split(
    X_train_smote, y_train_encoded, test_size=0.2, random_state=SEED, stratify=y_train_smote
)

# 10. Build DNN Model (Tanpa Self-Attention)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# 11. Train Model with EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train_main, y_train_main,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=50,
    callbacks=[early_stop],
    verbose=1
)

# 12. Evaluate on Test Set
y_pred_prob = model.predict(X_test)
y_pred_class = np.argmax(y_pred_prob, axis=1)
y_true_class = np.argmax(y_test_encoded, axis=1)

# 13. Metrik Evaluasi
print("\n=== Classification Report (Macro) Model DNN + SMOTE===")
print(classification_report(y_true_class, y_pred_class, digits=4))
roc_auc = roc_auc_score(y_test_encoded, y_pred_prob, average='macro', multi_class='ovr')
print(f"\nMacro-average ROC AUC: {roc_auc:.4f}")


Mounted at /content/drive
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8207/8207 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.4836 - loss: 1.0003 - val_accuracy: 0.4939 - val_loss: 0.9902
Epoch 2/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.4900 - loss: 0.9939 - val_accuracy: 0.4915 - val_loss: 0.9892
Epoch 3/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.4905 - loss: 0.9929 - val_accuracy: 0.4913 - val_loss: 0.9890
Epoch 4/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.4906 - loss: 0.9923 - val_accuracy: 0.4920 - val_loss: 0.9886
Epoch 5/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.4900 - loss: 0.9919 - val_accuracy: 0.4913 - val_loss: 0.9882
Epoch 6/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.4909 - loss: 0.9914 - val_accuracy: 0.4919 - val_loss: 0.9876
Epoch 7/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.4911 - loss: 0.9910 - val_accuracy: 0.4918 - val_loss: 0.9872
Epoch 8/100
8207/8207 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.4908 - loss: 0.990